In [ ]:
import pandas as pd

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

In [2]:
transript = """嗨您好
我這裡是信安保金代
我在5月30號的時候
有幫我的客戶投保了一張傷害險
但是我在6月12號的時候
收到一張照會單
跟我說
因為累積同業傷害險的保額
超過該個險種的個人年收入的倍數
所以我在賦約那個Y500R2的
那個限額投保275萬
叫我要補上契約變更書
調降那個賦約裡面的投保保額
想跟你確認一下這件事是怎麼了
好您好
那個我想先跟你核對一下身份
請問您在信安保金代的銷售人編號是
AR524403
好的
那再跟您請教一下
這一張保單的編號是
MA44632
好的
我看一下這張保單的狀況
嗯
這邊我們在系統上看到
因為您的客戶已經在其他保險公司同業
投保過了傷害險
所以才會有這個問題
可是按照我客戶的年收入
他可以投保的最高理賠額度是975萬
嗯是的沒錯
但是我們在授權公會上面
看到他在同業已經投保了700萬的傷害險
所以在我們這邊只能提供275萬傷害險的額度
那這邊可能要請您跟客戶確認一下
他是不是已經在哪一家保險公司有投保過
嗨那請問一下
你這邊有辦法透露是哪一家保險公司嗎
嗯不好意思
沒有辦法
因為根據各自法的關係
我們不能透露客戶詳細的投保資訊
可能要請您客戶這邊
自己找他原本的投保公司來申請投保資訊
或者是投保記錄
好的謝謝你
我來跟我的客戶確認一下
好謝謝
拜拜
拜拜
"""

In [ ]:
creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:500,
    GenParams.MIN_NEW_TOKENS:1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["<eof>"],
}

llm = Model("meta-llama/llama-2-70b-chat",creds, params,project_id)
# llm = Model("ibm/granite-13b-chat-v2",creds, params,project_id)


In [3]:
import sys
import logging
import os
import tempfile
import pathlib
import time
from concurrent.futures import ProcessPoolExecutor
from tqdm.notebook import tqdm

from dotenv import load_dotenv

from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader, PyPDFium2Loader
from sentence_transformers import SentenceTransformer

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM


from typing import Literal, Optional, Any
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Milvus, FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
import numpy as np
import pandas as pd

In [44]:
from dotenv import load_dotenv

from genai.client import Client
from genai.credentials import Credentials
from genai.schema import TextGenerationParameters, TextGenerationReturnOptions
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    ModerationHAP,
    ModerationHAPInput,
    ModerationHAPOutput,
    ModerationParameters,
    TextGenerationParameters,
)

load_dotenv()
client = Client(credentials=Credentials.from_env())

model = LangChainInterface(
    model_id="meta-llama/llama-2-70b-chat",
    # model_id="mistralai/mixtral-8x7b-instruct-v0-1",
    client=Client(credentials=Credentials.from_env()),
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.GREEDY,
        max_new_tokens=2000,
        min_new_tokens=1,
        # temperature=0.05,
        top_k=50,
        top_p=1,
        # stop_sequences=['```'],
    )
)

In [45]:
from IPython.display import display, Markdown, HTML, clear_output

prompt = f"""[INST]你是人壽客服中心分析人員，請總結以下對話紀錄，並提取關鍵信息。
    -識別對話人員身分，並在稍後的紀錄恰當描述對方身分。
    -請適當指稱對話人員，對話的人不一定是客戶，客戶在人壽行業有特殊定義。
    -使用"speaker 1"指稱，不要用“客戶”這個指稱。
    -speaker 2是全球人壽照會客服人員。
    <<SYS>>
    對話紀錄: `{transript}`
    <</SYS>>
    -請生成對話紀錄的總結。
    -識別對方的主要訴求，推薦跟進事項。
    -識別這個對話是關於什麼產品，提取銷售人員編號，提取保單編號。
    -識別對方的情緒。
    -跟進事項請明確(誰跟進，需要跟進的任務，目標日期)。
    -請生成markdown，適當分行顯示。
    [/INST]生成結果 in markdown:```"""

    # 格式:`
    # 對話紀錄總結：

    # 對話人員身分：

    # 主要訴求：

    # 產品：

    # 保單編號：

    # 銷售人員編號：

    # 情緒：

    # 跟進事項：
    # `

answer = model.invoke(prompt)

display(Markdown(answer))


對話紀錄:
================

對話人員:
---------------

* Speaker 1: 信安保金代的銷售人員 (AR524403)
* Speaker 2: 全球人壽照會客服人員

主要訴求:
---------------

* 信安保金代的客戶投保了一張傷害險，但是在其他保險公司已經投保過了傷害險，導致現在的投保額度僅能提供275萬
* 客戶的年收入可以投保的最高理賠額度是975萬，但是因為已經投保過傷害險，所以只能提供275萬的額度
* 銷售人員需要跟客戶確認一下他是不是已經在哪一家保險公司有投保過
* 全球人壽照會客服人員不能透露客戶詳細的投保資訊，可能要請您客戶這邊
* 銷售人員需要跟客戶確認一下他的投保記錄或者是投保公司

識別對方的情緒:
-------------------------

* 銷售人員的情緒不明確，但是可能是對客戶的調查和確認
* 全球人壽照會客服人員的情緒也不明確，但是可能是對客戶的調查和確認

跟進事項:
--------------

* 銷售人員需要跟客戶確認一下他是不是已經在哪一家保險公司有投保過
* 銷售人員需要跟客戶確認一下他的投保記錄或者是投保公司
* 全球人壽照會客服人員需要跟銷售人員聯繫，以確保客戶的投保資訊是正確的

識別產品:
--------------

* 傷害險

提取銷售人員編號:
---------------------------

* AR524403

提取保單編號:
---------------------

* MA44632

目標日期:
-------------

* 未提及

請注意，這只是一個機器翻譯的結果，可能存在一些偏差或錯誤。

In [ ]:
def buildprompt(text):
    prompt = f"""[INST]
    you are call center specialist, need to summarize the transcript in 200 words:
    -generate summary as json.
    -base on the summary json format provided.
    -dont generate note.
    -show json only.
    -rating in 5 class (good, bad, )
    -end with <eof>
    transcript: `{text}`
    summary json format:`{{
        总结:,
        意图:,
        产品:,
        情绪:,
    }}`
    [/INST]summary json:"""
    return prompt

In [ ]:
def buildpromptgetlabels(text):
    prompt = f"""[INST]
    you are call center specialist, need to label the transcript in 200 words:
    -generate summary as json.
    -base on the summary json format provided.
    -dont generate note.
    -dont generate explaination.
    -show json only.
    -generate 10 labels for the user and product make most sense.
    -end with <eof>
    transcript: `{text}`
    summary json format:`{{
        labels:
    }}`
    [/INST]summary json:"""
    return prompt

In [ ]:
sid = 1728970632164929538
conversation = pd.read_csv('conversation_log.csv')
subset = conversation[conversation['sid']==sid]
subset = subset[['role','conversation']]
subset.head()
ajson = subset.to_json(orient='records',force_ascii=False)
# ajson
print(f"\n客户对话{sid}分析")
prompt = buildprompt(ajson[:4000])
for response in llm.generate_text([prompt]):
    print(response)
print(f"\n客户对话{sid}标签")
prompt = buildpromptgetlabels(ajson[:4000])
for response in llm.generate_text([prompt]):
    print(response)

In [ ]:
conversation = pd.read_csv('conversation_log.csv')
sids = conversation['sid'].unique()
for sid in sids:
    subset = conversation[conversation['sid']==sid]
    subset = subset[['role','conversation']]
    # subset.head()
    # ajson = subset.to_csv()
    # print(ajson)
    ajson = subset.to_json(orient='records',force_ascii=False)
    # ajson
    # print(f"\n客户对话{sid}分析")
    # prompt = buildprompt(ajson[:4000])
    # for response in llm.generate_text([prompt]):
    #     print(response)
    print(f"\n客户对话{sid}标签")
    prompt = buildpromptgetlabels(ajson[:4000])
    for response in llm.generate_text([prompt]):
        print(response)